***Optimization of Wafer dataset with XGBoost***

In [ ]:
!pip install optuna
!pip install xgboost

     |████████████████████████████████| 308 kB 5.3 MB/s 
     |████████████████████████████████| 209 kB 47.8 MB/s 
     |████████████████████████████████| 80 kB 8.7 MB/s 
     |████████████████████████████████| 75 kB 4.0 MB/s 
     |████████████████████████████████| 112 kB 72.3 MB/s 
     |████████████████████████████████| 49 kB 5.5 MB/s 
     |████████████████████████████████| 149 kB 54.4 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=15a84a42ff7b85500685f88d3f25e46c1bc8ca6e39b326b7e8dc1451aa766494
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import numpy as np
import pandas as pd 
import optuna
import xgboost as xgb 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('wafer_16012020_051629.csv')

In [ ]:
x= df.drop(columns=['Unnamed: 0' , 'Good/Bad'])

In [ ]:
for (columnName, columnData) in x.iteritems():
    x[columnName]= x[columnName].fillna(x[columnName].mean())

In [ ]:
y = df['Good/Bad']
y

0     1
1     1
2     1
3     1
4     1
     ..
95   -1
96   -1
97   -1
98   -1
99   -1
Name: Good/Bad, Length: 100, dtype: int64

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
sta_sca = StandardScaler()
x = sta_sca.fit_transform(x)

In [ ]:
train_x , test_x , train_y , test_y= train_test_split(x , y , test_size = .20 )

In [ ]:
def objective_classification(trial) :
    train_x , test_x , train_y , test_y= train_test_split(x , y , test_size = .20 )
    param  = {
        #'tree_method' :'approx',
        
        'sketch_eps' :trial.suggest_float('sketch_eps',0.01,1.0), ## Only used for 'approx' tree method, consider setting low number for accuracy 
        
        'verbosity':0, ##print more info on tree buliding process
        
        'validate_parameters':'True', ## performs validation to check whether parameter exists or not.

        'n_estimators' : 20000,
        
        #'nthread' : 10, ## No. of parallel threads to run Xgboost
        
        'max_depth' :trial.suggest_int('max_depth',1,20), ## Higher the depth of value more complex and ovefitting could occur
        
        'booster':trial.suggest_categorical('booster',['dart','gbtree','gblinear']), ## Dart and gbree for tree based booster method. gblinear used for linear function.
        
        'base_score' :trial.suggest_float('base_score' , .1 , .9), ## initial prediction score of all instances
        
        'eval_metric' :'logloss', ## Added as objective used as evaluation  metrics for validation data, multiple metrics can be passed in a list for evaluation.
        
        'seed' :5, ## Random number seed
        
        'objective' : 'binary:logistic' , ## Used to pass learning objective to be applied on model to get desired result
        
        'lambda' : trial.suggest_float('lambda' , 1e-4 , 1.0), ## Model will become more conservative with increase in value. L2 regularization term on weights
        
        'alpha' :trial.suggest_float('alpha' , 1e-4 , 1.0), ## Model will become more conservative with increase in value. L1 regularization term on weights
        
        'subsample' : trial.suggest_float('subsample' , .1,.5), ## Occurs once in every bossting itration. Tuning value to 0.5 means that XGBoost would randomly sample half of the training data
        
        'colsample_bytree' : trial.suggest_float('colsample_bytree' , .5 ,1.0), ## Subsample ratio of columns when constructing each tree
        
        'min_child_weight' :trial.suggest_int('min_child_weight',1,10), ## Minimum sum of weights of all observations required in a child.Too high values can lead to under-fitting hence, it should be tuned using CV
        
        'max_delta_step' :trial.suggest_int('max_delta_step',1,10), ## Used when in logistic regression when class is extremely imbalanced. Tuning value between 1-10 help control the update.
        
        'sampling_method':trial.suggest_categorical('sampling_method',['uniform','gradient_based']), ## used to sample the training instances. set subsample >= 0.5 for good results.
        
        #'updater':'grow_colmaker', ## An advanced parameter that is usually set automatically, depending on some other parameters, used to run sequence of tree updaters 
        
        
    }
   
    if param['booster'] in ['dart','gbtree']:
      
      param['gamma'] : trail.suggest_float('gamma',1e-3,4.0) ## Gamma specifies the minimum loss reduction required to make a split. The values can vary depending on the loss function and should be tuned.
      
      param['eta'] : trail.suggest_float('eta',.01,0.2) ##Makes the model more robust by shrinking the weights on each step. Typical final values to be used: 0.01-0.2
      

    if param['booster'] == 'dart':
      param['sample_type']:trail.suggest_categorical('sample_type',['uniform','weighted']) ## uniform: dropped trees are selected uniformly.weighted: dropped trees are selected in proportion to weight.

      param['normalize_type']:trail.suggest_categorical('normalize_type',['tree','forest']) ## tree: new trees have the same weight of each of dropped trees. forest: new trees have the same weight of sum of dropped trees (forest).

      param['rate_drop'] :trail.suggest_float('rate_drop',0.0,1.0) ## Dropout rate and has range from 0.0 to 1.0.

      param['one_drop'] :0 ## Least one tree is always dropped during the dropout if this enabled

      param['skip_drop'] :trail.suggest_float('skip_drop',0.0,1.0) ## Probability of skipping the dropout procedure and has range from 0.0 to 1.0.

    if param['booster'] == 'gblinear':
      param['updater']:trail.suggest_categorical('updater',['shotgun','coord_descent']) ## Defines various algo to fit linear based model.
      param['feature_selector']:trail.suggest_categorical('feature_selector',['cyclic','shuffle','random','greedy','thrifty']) ## Defines various feature selection and ordering method.
        
    xgb_classification = xgb.XGBClassifier(**param)
    xgb_classification.fit(train_x , train_y,eval_set  = [(test_x, test_y)],eval_metric='logloss',verbose=True)
 
    pred = xgb_classification.predict(test_x)
    accuracy  = xgb_classification.score(test_x,test_y) 
    #
    return accuracy
        
        
    

In [ ]:
find_param = optuna.create_study(direction='minimize')

[I 2021-11-22 17:14:37,520] A new study created in memory with name: no-name-827a8cc7-9a32-4d1e-86c6-64dc9f0c28ea


In [ ]:
find_param.optimize(objective_classification  , n_trials=10)

Streaming output truncated to the last 5000 lines.
[15000]	validation_0-logloss:0.533755
[15001]	validation_0-logloss:0.533755
[15002]	validation_0-logloss:0.533755
[15003]	validation_0-logloss:0.533755
[15004]	validation_0-logloss:0.533755
[15005]	validation_0-logloss:0.533755
[15006]	validation_0-logloss:0.533755
[15007]	validation_0-logloss:0.533755
[15008]	validation_0-logloss:0.533755
[15009]	validation_0-logloss:0.533755
[15010]	validation_0-logloss:0.533755
[15011]	validation_0-logloss:0.533755
[15012]	validation_0-logloss:0.533755
[15013]	validation_0-logloss:0.533755
[15014]	validation_0-logloss:0.533755
[15015]	validation_0-logloss:0.533755
[15016]	validation_0-logloss:0.533755
[15017]	validation_0-logloss:0.533755
[15018]	validation_0-logloss:0.533755
[15019]	validation_0-logloss:0.533755
[15020]	validation_0-logloss:0.533755
[15021]	validation_0-logloss:0.533755
[15022]	validation_0-logloss:0.533755
[15023]	validation_0-logloss:0.533755
[15024]	validation_0-logloss:0.533755

[I 2021-11-22 17:17:09,810] Trial 0 finished with value: 0.7 and parameters: {'sketch_eps': 0.589973573588527, 'max_depth': 17, 'booster': 'gblinear', 'base_score': 0.24901955138128928, 'lambda': 0.40246051256855986, 'alpha': 0.3751384171852157, 'subsample': 0.292654291749962, 'colsample_bytree': 0.8002511874931637, 'min_child_weight': 9, 'max_delta_step': 8, 'sampling_method': 'gradient_based'}. Best is trial 0 with value: 0.7.


Streaming output truncated to the last 5000 lines.
[15000]	validation_0-logloss:0.491195
[15001]	validation_0-logloss:0.491195
[15002]	validation_0-logloss:0.491195
[15003]	validation_0-logloss:0.491195
[15004]	validation_0-logloss:0.491195
[15005]	validation_0-logloss:0.491195
[15006]	validation_0-logloss:0.491195
[15007]	validation_0-logloss:0.491195
[15008]	validation_0-logloss:0.491195
[15009]	validation_0-logloss:0.491195
[15010]	validation_0-logloss:0.491195
[15011]	validation_0-logloss:0.491195
[15012]	validation_0-logloss:0.491195
[15013]	validation_0-logloss:0.491195
[15014]	validation_0-logloss:0.491195
[15015]	validation_0-logloss:0.491195
[15016]	validation_0-logloss:0.491195
[15017]	validation_0-logloss:0.491195
[15018]	validation_0-logloss:0.491195
[15019]	validation_0-logloss:0.491195
[15020]	validation_0-logloss:0.491195
[15021]	validation_0-logloss:0.491195
[15022]	validation_0-logloss:0.491195
[15023]	validation_0-logloss:0.491195
[15024]	validation_0-logloss:0.491195

[I 2021-11-22 17:19:54,429] Trial 1 finished with value: 0.6 and parameters: {'sketch_eps': 0.09426341820735383, 'max_depth': 15, 'booster': 'gblinear', 'base_score': 0.49110007578994497, 'lambda': 0.03893374170900119, 'alpha': 0.11459848092774408, 'subsample': 0.12735049303811566, 'colsample_bytree': 0.646446674260736, 'min_child_weight': 7, 'max_delta_step': 3, 'sampling_method': 'gradient_based'}. Best is trial 1 with value: 0.6.


Streaming output truncated to the last 5000 lines.
[15000]	validation_0-logloss:0.585077
[15001]	validation_0-logloss:0.582227
[15002]	validation_0-logloss:0.581158
[15003]	validation_0-logloss:0.582751
[15004]	validation_0-logloss:0.581129
[15005]	validation_0-logloss:0.579798
[15006]	validation_0-logloss:0.580926
[15007]	validation_0-logloss:0.579793
[15008]	validation_0-logloss:0.581211
[15009]	validation_0-logloss:0.58
[15010]	validation_0-logloss:0.58173
[15011]	validation_0-logloss:0.584624
[15012]	validation_0-logloss:0.585666
[15013]	validation_0-logloss:0.583991
[15014]	validation_0-logloss:0.584799
[15015]	validation_0-logloss:0.582421
[15016]	validation_0-logloss:0.582779
[15017]	validation_0-logloss:0.582804
[15018]	validation_0-logloss:0.58249
[15019]	validation_0-logloss:0.581566
[15020]	validation_0-logloss:0.584622
[15021]	validation_0-logloss:0.585937
[15022]	validation_0-logloss:0.587065
[15023]	validation_0-logloss:0.587479
[15024]	validation_0-logloss:0.590559
[1502

...

[I 2021-11-22 17:49:21,122] Trial 7 finished with value: 0.75 and parameters: {'sketch_eps': 0.736236413770203, 'max_depth': 3, 'booster': 'gbtree', 'base_score': 0.44603685911607516, 'lambda': 0.8222280568259852, 'alpha': 0.5544048924373948, 'subsample': 0.2073163894923863, 'colsample_bytree': 0.7993374789772383, 'min_child_weight': 8, 'max_delta_step': 4, 'sampling_method': 'uniform'}. Best is trial 6 with value: 0.55.


Streaming output truncated to the last 5000 lines.
[15000]	validation_0-logloss:0.60878
[15001]	validation_0-logloss:0.609463
[15002]	validation_0-logloss:0.60861
[15003]	validation_0-logloss:0.606695
[15004]	validation_0-logloss:0.608769
[15005]	validation_0-logloss:0.604571
[15006]	validation_0-logloss:0.608219
[15007]	validation_0-logloss:0.60825
[15008]	validation_0-logloss:0.605587
[15009]	validation_0-logloss:0.603847
[15010]	validation_0-logloss:0.604758
[15011]	validation_0-logloss:0.603163
[15012]	validation_0-logloss:0.607726
[15013]	validation_0-logloss:0.609903
[15014]	validation_0-logloss:0.609343
[15015]	validation_0-logloss:0.607275
[15016]	validation_0-logloss:0.605456
[15017]	validation_0-logloss:0.604742
[15018]	validation_0-logloss:0.606421
[15019]	validation_0-logloss:0.60528
[15020]	validation_0-logloss:0.605136
[15021]	validation_0-logloss:0.605966
[15022]	validation_0-logloss:0.606558
[15023]	validation_0-logloss:0.60759
[15024]	validation_0-logloss:0.604611
[150

[I 2021-11-22 17:52:00,436] Trial 8 finished with value: 0.65 and parameters: {'sketch_eps': 0.7336263363050943, 'max_depth': 10, 'booster': 'gbtree', 'base_score': 0.809399814640657, 'lambda': 0.23922621488654902, 'alpha': 0.006120677605206578, 'subsample': 0.48730805081184303, 'colsample_bytree': 0.9146555803161849, 'min_child_weight': 5, 'max_delta_step': 8, 'sampling_method': 'gradient_based'}. Best is trial 6 with value: 0.55.


Streaming output truncated to the last 5000 lines.
[15000]	validation_0-logloss:0.705946
[15001]	validation_0-logloss:0.70525
[15002]	validation_0-logloss:0.701666
[15003]	validation_0-logloss:0.701991
[15004]	validation_0-logloss:0.705703
[15005]	validation_0-logloss:0.706431
[15006]	validation_0-logloss:0.705045
[15007]	validation_0-logloss:0.701997
[15008]	validation_0-logloss:0.700291
[15009]	validation_0-logloss:0.707322
[15010]	validation_0-logloss:0.709255
[15011]	validation_0-logloss:0.716246
[15012]	validation_0-logloss:0.717123
[15013]	validation_0-logloss:0.712375
[15014]	validation_0-logloss:0.712376
[15015]	validation_0-logloss:0.710162
[15016]	validation_0-logloss:0.706942
[15017]	validation_0-logloss:0.708923
[15018]	validation_0-logloss:0.70601
[15019]	validation_0-logloss:0.710937
[15020]	validation_0-logloss:0.710537
[15021]	validation_0-logloss:0.709431
[15022]	validation_0-logloss:0.71613
[15023]	validation_0-logloss:0.719606
[15024]	validation_0-logloss:0.723827
[1

[I 2021-11-22 17:54:36,746] Trial 9 finished with value: 0.6 and parameters: {'sketch_eps': 0.690647757266719, 'max_depth': 7, 'booster': 'gbtree', 'base_score': 0.246174360049217, 'lambda': 0.5874900260553897, 'alpha': 0.6546187115152561, 'subsample': 0.38851820392707603, 'colsample_bytree': 0.7899246829373207, 'min_child_weight': 8, 'max_delta_step': 6, 'sampling_method': 'gradient_based'}. Best is trial 6 with value: 0.55.


In [ ]:
xg_claf=xgb.XGBClassifier(**best_fit)
xg_claf.fit(train_x,train_y)

XGBClassifier(alpha=0.18545302115796997, base_score=0.8032138788332793,
              colsample_bytree=0.7206580950463729, lambda=0.6845072442297875,
              max_delta_step=10, max_depth=18, min_child_weight=10,
              sampling_method='gradient_based', sketch_eps=0.09739787748673387,
              subsample=0.18854532336250285)

In [ ]:
print("Accuracy:")
xg_claf.score(test_x,test_y)

Accuracy:


0.6